In [50]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

def load_matrix(data, matrix_size=200):
    # Filter interactions within the same chromosome
    data = data[data['chr1'] == data['chr2']]
    # Determine the range of positions
    max_pos = data[['pos1', 'pos2']].max().max()
    min_pos = data[['pos1', 'pos2']].min().min()
    # Define the bin size dynamically based on the desired matrix size
    bin_size = (max_pos - min_pos) / matrix_size
    # Create bins
    bins = np.linspace(min_pos, max_pos, matrix_size + 1)
    # Create a mapping of positions to bins
    data['bin1'] = np.digitize(data['pos1'], bins) - 1
    data['bin2'] = np.digitize(data['pos2'], bins) - 1
    # Ensure bins do not exceed the matrix size
    data['bin1'] = data['bin1'].clip(upper=matrix_size - 1)
    data['bin2'] = data['bin2'].clip(upper=matrix_size - 1)
    # Create unique identifiers for each bin
    data['bin_id1'] = data['chr1'].astype(str) + '_' + data['bin1'].astype(str)
    data['bin_id2'] = data['chr2'].astype(str) + '_' + data['bin2'].astype(str)
    # Get unique bins
    unique_bins = np.unique(np.concatenate([data['bin_id1'], data['bin_id2']]))
    # Initialize the contact matrix
    contact_matrix = np.zeros((matrix_size, matrix_size))
    # Populate the contact matrix
    for _, row in data.iterrows():
        idx1 = row['bin1']
        idx2 = row['bin2']
        contact_matrix[idx1, idx2] += row['interaction']
        contact_matrix[idx2, idx1] += row['interaction']  # Assuming symmetry
    
    return contact_matrix

def find_max_shape_matrix(genotype: str) -> str:
    addr = ""
    max = (0, 0)
    for file in os.listdir(genotype):
        filepath = os.path.join(genotype, file)
        full = pd.read_csv(filepath, sep='\t', header=None)
        full.columns = ['chr1', 'pos1', 'chr2', 'pos2', 'interaction']
        # Find unique values in column 2 (pos1) and column 4 (pos2)
        full_posx = full['pos1'].unique()
        full_posy = full['pos2'].unique()
        if len(full_posx) > max[0] and len(full_posy) > max[1]:
            max = (len(full_posx), len(full_posy))
            addr = filepath

    return addr

def augment(genotype:str, filepaths:list[str]) -> list[pd.DataFrame]:
    """
    genotype: str, the genotype directory, eg. "./GM12878"
    filepaths: list[str], the list of filepaths of the matrices in the genotype directory
    """
    augmented_matrices = [] # Stores all augmented matrices

    max_size_file_path = find_max_shape_matrix(genotype)

    # Find the reference axes
    reference_data = pd.read_csv(max_size_file_path, sep='\t', header=None)
    # Assign column names
    reference_data.columns = ['chr1', 'pos1', 'chr2', 'pos2', 'interaction']
    # Find unique values in column 2 (pos1) and column 4 (pos2)
    unique_posx = reference_data['pos1'].unique()
    unique_posy = reference_data['pos2'].unique()

    # Find all (200, 200) full matrices
    full_matrices_dir = []
    for file in os.listdir(genotype):
        filepath = os.path.join(genotype, file)
        if os.path.isfile(filepath):
            matrix = load_matrix(pd.read_csv(filepath, sep='\t', header=None, names=['chr1', 'pos1', 'chr2', 'pos2', 'interaction']))
            if matrix.shape == (200, 200):
                full_matrices_dir.append(filepath)
                augmented_matrices.append(matrix)

    # Find all matrices that are not 200x200
    not_full_matrices = []
    for filepath in filepaths:
        if filepath not in full_matrices_dir:
            not_full_matrices.append(filepath)

    for filepath in not_full_matrices:
        temp_data = pd.read_csv(filepath, sep='\t', header=None)
        temp_data.columns = ['chr1', 'pos1', 'chr2', 'pos2', 'interaction']
        temp_x = temp_data['pos1'].unique()
        temp_y = temp_data['pos2'].unique()
        matrix = load_matrix(pd.read_csv(filepath, sep='\t', header=None, names=['chr1', 'pos1', 'chr2', 'pos2', 'interaction']))
        diff_x = np.setdiff1d(unique_posx, temp_x)  # columns missing
        diff_y = np.setdiff1d(unique_posy, temp_y)  # rows missing
        missing_data = reference_data[reference_data['pos1'].isin(diff_x) | reference_data['pos2'].isin(diff_y)]
        aug_data = pd.concat([temp_data, missing_data], ignore_index=True)
        aug_matrix = load_matrix(aug_data)
        augmented_matrices.append(aug_matrix)
    return augmented_matrices

# Function to load all matrices from a directory
def load_matrices(directory, contact_matrices, filepaths):
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            filepath = os.path.join(directory, filename)
            # matrix = load_contact_matrix(filepath)
            matrix = load_matrix(pd.read_csv(filepath, sep='\t', header=None, names=['chr1', 'pos1', 'chr2', 'pos2', 'interaction']))
            contact_matrices.append(matrix)
            filepaths.append(filepath)

# Directories
directories = {
    'GM12878': './GM12878',
    'HAP1': './HAP1',
    'Hela': './Hela',
    'K562': './K562'
}

# Load all matrices and labels
contact_matrices = []
labels = []

for label, directory in directories.items():
    load_matrices(directory, contact_matrices, labels, label)

contact_matrices = np.array(contact_matrices)
labels = np.array(labels)

print(f"Total matrices loaded: {contact_matrices.shape[0]}")  # Debug print
print(f"Total labels loaded: {labels.shape[0]}")  # Debug print

# Encode the labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels_encoded), y=labels_encoded)
class_weights_dict = dict(enumerate(class_weights))

print("Class weights:", class_weights_dict)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(contact_matrices, labels_categorical, test_size=0.2, random_state=42)

# Reshape the data to add the channel dimension
X_train = X_train.reshape(X_train.shape[0], 200, 200, 1)
X_val = X_val.reshape(X_val.shape[0], 200, 200, 1)


Total matrices loaded: 2611
Total labels loaded: 2611
Class weights: {0: 27.197916666666668, 1: 0.7118320610687023, 2: 0.40243526510480887, 3: 13.598958333333334}


In [51]:
# Initialize ImageDataGenerator with augmentation for the training data
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Use the generator to augment the training data
train_generator = datagen.flow(X_train, y_train, batch_size=32)

# Use a separate generator for the validation data without augmentation
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
validation_generator = validation_datagen.flow(X_val, y_val, batch_size=32)



In [53]:
from tensorflow.keras.regularizers import l2
# Define a simplified CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 1), kernel_regularizer=l2(0.001)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),
    
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),
    
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


/Users/rogersyang/Documents/PhD Application/Research/3D DNA Structure Heterogeneity (Elena)/DNA3D/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_11 (Conv2D)              │ (None, 198, 198, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 97, 97, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 147456)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 128)            │    18,874,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,893,828 (72.07 MB)

 Trainable params: 18,893,828 (72.07 MB)

 Non-trainable params: 0 (0.00 B)

In [54]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

# Train the model with class weights and learning rate scheduler
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    class_weight=class_weights_dict,
    callbacks=[reduce_lr]
)


Epoch 1/20


/Users/rogersyang/Documents/PhD Application/Research/3D DNA Structure Heterogeneity (Elena)/DNA3D/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


66/66 ━━━━━━━━━━━━━━━━━━━━ 18s 257ms/step - accuracy: 0.3520 - loss: 1.7005 - val_accuracy: 0.0000e+00 - val_loss: 1.4157 - learning_rate: 0.0010
Epoch 2/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 17s 252ms/step - accuracy: 0.0473 - loss: 1.4330 - val_accuracy: 0.0000e+00 - val_loss: 1.4056 - learning_rate: 0.0010
Epoch 3/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 17s 252ms/step - accuracy: 0.0098 - loss: 1.3981 - val_accuracy: 0.0000e+00 - val_loss: 1.4066 - learning_rate: 0.0010
Epoch 4/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 17s 254ms/step - accuracy: 0.0082 - loss: 1.3620 - val_accuracy: 0.0000e+00 - val_loss: 1.4081 - learning_rate: 0.0010
Epoch 5/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 17s 253ms/step - accuracy: 0.0112 - loss: 1.4292 - val_accuracy: 0.0000e+00 - val_loss: 1.4095 - learning_rate: 0.0010
Epoch 6/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 17s 252ms/step - accuracy: 0.0149 - loss: 1.6338 - val_accuracy: 0.0000e+00 - val_loss: 1.4158 - learning_rate: 0.0010
Epoch 7/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 17s 252ms/step - accuracy: 0.00

In [63]:
# Initialize ImageDataGenerator with more aggressive augmentation for the training data
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Use the generator to augment the training data
train_generator = datagen.flow(X_train, y_train, batch_size=32)

# Evaluate the model on the validation set
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

# Function to classify a new matrix
def classify_matrix(matrix, model, label_encoder):
    # Ensure the matrix is the correct shape
    matrix = matrix.reshape(1, 200, 200, 1)  # Add batch and channel dimensions
    matrix = matrix / 255.0  # Normalize
    prediction = model.predict(matrix)
    class_index = np.argmax(prediction, axis=1)
    class_label = label_encoder.inverse_transform(class_index)
    return class_label[0]

# Example usage with a new matrix
matrix = load_contact_matrix('./Hela/ml1_CACGACCT-CGTTACTT.txt')
normalized_matrix = z_score_normalize(matrix)
extended_matrix = extend_matrix(normalized_matrix)
class_label = classify_matrix(extended_matrix, model, label_encoder)
print(f'The new matrix belongs to the collection: {class_label}')


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.0000e+00 - loss: 1.4492
Validation Accuracy: 0.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
The new matrix belongs to the collection: GM12878
